### 事件的声明

- 事件的声明
    - 完整声明
    - 简略声明（字段式声明，field-like）
- 有了委托字段/属性，为什么还需要事件
    - 为了程序的逻辑更加 "有道理" 更加安全，<font color="lightGreen">谨防借刀杀人</font>
- 所以事件的本质是委托字段的一个包装器
    - 这个包装器对委托字段的访问起限制作用，相当于一个"蒙板"
    - 封装（encapsulation）的一个重要功能就是隐藏
    - 事件对外界隐藏了委托实例的大部分功能，<font color="lightGreen">仅暴露添加/移除事件处理器的功能</font>
    - 添加/移除事件处理器的时候可以直接使用方法名，这是委托实例所不具备的功能
- 用于声明事件的委托类型的命名约定
    - 用于声明Foo事件的委托，一般命名为<font color="lightGreen">FooEventHandler</font>（除非是一个非常通用的事件约束）
    - FooEventHandler委托的参数一般有两个
        - 第一个是object类型，名为sender，实际上就是<font color="lightGreen">事件的拥有者</font>，事件的source
        - 第二个是EventArgs类的派生类，类名一般为FooEventArgs, 参数名为e。也就是<font color="lightGreen">事件参数</font>
        - 虽然没有官方的说法，但我们可以把<font color="lightGreen">委托的参数列表看做是事件发生后发送给是事件响应者的"事件消息"</font>
    - 触发Foo事件的方法一般命名为OnFoo, 即"因何引发"、"事出有因"
        - 访问级别为protected, 不能为public
- 事件的命名约定
    - 带有时态的动词或动词短语
    - 事件拥有者"正在做"什么事情，用进行时；事件拥有者"做完了"什么事情，用完成时


#### 例1 —— 事件的完整声明

In [ ]:
using System.Threading;

public delegate void OrderEventhandler(Customer customer, OrderEventArgs e); //用于声明事件的委托

public class OrderEventArgs:EventArgs //事件参数，继承EventArgs基类
{
    public string DishName {get;set;}
    public string Size {get;set;}
}


public class Customer //事件拥有者
{
    private OrderEventhandler orderEventhandler;

    public event OrderEventhandler Order
    {
        add{
            orderEventhandler += value;
        }
        remove{
            orderEventhandler -= value;
        }
    }

    
    public double Bill{get;set;}

    public void PayTheBill()
    {
        $"I will pay ${Bill}".Display();
    }

    public void WalkIn()
    {
        "Walk into the restaurant.".Display();
    }
    public void SitDown()
    {
        "Sit Down".Display();
    }
    public void Think()
    {
        for(int i = 0; i < 5; i++){
            "Let me think ...".Display();
            Thread.Sleep(1000);
        }

        if(this.orderEventhandler != null)
        {
            OrderEventArgs e = new OrderEventArgs();
            e.DishName = "Kongpao Chicken";
            e.Size = "large";
            orderEventhandler.Invoke(this,e);
        }
    }

    public void Action()
    {
        WalkIn();
        SitDown();
        Think();
    }

}

public class Waiter
{
    public void Action(Customer customer, OrderEventArgs e)
    {
        $"I will serve you the dish - {e.DishName}".Display();
        double price = 10;
        switch(e.Size)
        {
            case "small":
                price = price * 0.5;
                break;
            case "large":
                price = price * 1.5;
                break;
        }
        customer.Bill += price;
    }
}

Customer Customer1 = new Customer();
Waiter waiter = new Waiter();
Customer1.Order += waiter.Action;
Customer1.Action();
Customer1.PayTheBill();

#### 例2 ——— 委托的简约声明

In [ ]:
using System.Threading;

public delegate void OrderEventhandler(Customer customer, OrderEventArgs e); //用于声明事件的委托

public class OrderEventArgs:EventArgs //事件参数，继承EventArgs基类
{
    public string DishName {get;set;}
    public string Size {get;set;}
}


public class Customer //事件拥有者
{
    // private OrderEventhandler orderEventhandler;

    // public event OrderEventhandler Order
    // {
    //     add{
    //         orderEventhandler += value;
    //     }
    //     remove{
    //         orderEventhandler -= value;
    //     }
    // }
    public event OrderEventhandler Order;
    
    public double Bill{get;set;}

    public void PayTheBill()
    {
        $"I will pay ${Bill}".Display();
    }

    public void WalkIn()
    {
        "Walk into the restaurant.".Display();
    }
    public void SitDown()
    {
        "Sit Down".Display();
    }
    public void Think()
    {
        for(int i = 0; i < 5; i++){
            "Let me think ...".Display();
            Thread.Sleep(1000);
        }

        OnOrder("Kongpao Chicken", "large");
    }

    protected void OnOrder(string dishName, string size)
    {
        if(Order != null)
        {
            OrderEventArgs e = new OrderEventArgs();
            e.DishName = dishName;
            e.Size = size;
            Order.Invoke(this,e);
        }
    }

    public void Action()
    {
        WalkIn();
        SitDown();
        Think();
    }

}

public class Waiter
{
    public void Action(Customer customer, OrderEventArgs e)
    {
        $"I will serve you the dish - {e.DishName}".Display();
        double price = 10;
        switch(e.Size)
        {
            case "small":
                price = price * 0.5;
                break;
            case "large":
                price = price * 1.5;
                break;
        }
        customer.Bill += price;
    }
}

Customer Customer1 = new Customer();
Waiter waiter = new Waiter();
Customer1.Order += waiter.Action;
Customer1.Action();
Customer1.PayTheBill();

#### 为了解决委托在类的外部被滥用的问题，引入了事件

### 事件与委托的关系

- 事件真的是"以特殊方式声明的委托字段/实例吗"？
    - 不是！知识声明的时候看起来像（对比委托字段与事件的简化声明，field-like）
    - 事件声明的时候使用了委托类型，简化声明造成事件看上去像是一个委托的字段（实例），而event关键字则更像是一个修饰符——这就是错觉来源之一
    - 订阅事件的时候+=操作符后面可以是一个委托实例，这与委托实例的赋值方法语法相同，这也让事件看起来像是一个委托字段——这是错觉的又一来源
    - 重申：事件的本质是假装在委托字段上的一个蒙板（mask），是个起掩蔽作用的包装器。这个用于阻挡非法操作的"蒙板"绝不是委托字段本身
- 为什么要使用委托类型来声明事件？
    - 站在source的角度来看，是为了表明source能对外界传递那些消息
    - 站在subscriber的角度来看，它是一种约定，是为了约束能够使用什么样的签名方法来处理（响应）事件
    - 委托类型的实例将用于存储（引用）事件处理器
- 对比事件与属性
    - 属性不是字段——很多时候属性是字段的包装器，这个包装器是用来保护字段不被滥用
    - 事件不是委托字段——他是委托字段的包装器，这个包装器用来保护委托字段不被滥用
    - 包装器永远都不可能是被包装的东西